In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch

# Problema `OR`

In [ ]:
train_df = pd.read_csv('OR_trn.csv', index_col=None, header=None)

In [ ]:
train_df

In [ ]:
# Extraigo patrones y etiquetas

X = train_df.iloc[:,:-1].to_numpy()
Y = train_df.iloc[:,-1].to_numpy()

In [ ]:
# Convierto a Tensores

X = torch.from_numpy(X)
Y = torch.from_numpy(Y)

In [ ]:
# Dibujo los patrones

fig, ax = plt.subplots(1, 1, figsize=(6,6))

ax.scatter(X[Y==-1,0], X[Y==-1,1], s=20, c='C0', marker='x', label='-1')
ax.scatter(X[Y==1,0], X[Y==1,1], s=20, c='C1', marker='x', label='1')
ax.legend(('1','-1'), loc='best', title="Clases")
ax.set_xlabel('$X_{1}$', fontsize=14)
ax.set_ylabel('$X_{2}$', fontsize=14)

ax.grid(True)

### Construyo una clase `Perceptrón`

In [ ]:
class Perceptron():
    
    #===========================================
    def __init__(self, Ndim=2, lr=0.005):
        
        self.lr = lr
        
        self.W = torch.rand(Ndim + 1) - 0.5  # Inicializo en el rango [-0.5, 0.5]
        
        self.grad = torch.zeros_like(self.W)
    
    
    #===========================================
    def forward(self, x):
        
        x = torch.hstack( (torch.tensor(-1.), x) )  # [1.0 ...]
        
        yp = torch.sum(x * self.W)
        
        return yp
    
    
    #===========================================
    def backward(self, x, y):
        
        yp = self.forward(x)
        
        x = torch.hstack( (torch.tensor(-1.), x) )  # [1.0 ...]
        
        self.grad = 2 * (y - yp) * x
    
    #===========================================
    def update_w(self):
        
        # Actualizo pesos
        self.W += self.lr * self.grad
        
        #self.W /= torch.norm(self.W)  # REGULARIZO
        
        # Borro gradiente
        self.grad.fill_(0.)

In [ ]:
# Instancio un perceptrón

perceptron = Perceptron()

In [ ]:
yp = perceptron.forward(X[0,:])

print(f'\nSalida deseada: {Y[0]}')
print(f'Salida predicha: {yp}\n')

In [ ]:
print(f'\nGradiente inicial: {perceptron.grad}')

perceptron.backward(X[0,:], Y[0])

print(f'Gradiente actualizado: {perceptron.grad}\n')

In [ ]:
print(f'\nPesos iniciales: {perceptron.W} -- Gradiente: {perceptron.grad}\n')

perceptron.update_w()

print(f'Pesos actualizados: {perceptron.W} -- Gradiente: {perceptron.grad}\n')

---

## Entrenamiento

In [ ]:
perceptron = Perceptron(lr = 0.01)

for epoca in range(10):
    
    loss = 0
    Acc = 0
    
    counter = 1
    for i in range(X.shape[0]):
        
        x = X[i,:]
        y = Y[i]
        
        yp = perceptron.forward(x)
        
        if torch.sign(yp * y) > 0:
            Acc += 1
        
        loss += (y - yp)**2
        
        perceptron.backward(x, y)
        perceptron.update_w()
        
        #print(f'Loss: {loss.item()/counter} -- Acc: {Acc/counter}')
        
        #counter += 1
    
    print('\n======================================\n')
    print(f'Epoca: {epoca} -- Loss: {loss.item()/X.shape[0]:.4} -- Acc: {Acc/X.shape[0]}')

In [ ]:
def boundary(W, x):
    '''
    Esta función devuelve la salida sobre la frontera de decisión.
    '''
    
    w0 = W[0]
    w1 = W[1]
    w2 = W[2]
    
    y = w0/w2 - w1/w2 * x
    
    return y

In [ ]:
# DIBUJO PATRONES Y FROTERA DE DECISION

fig, ax = plt.subplots(1, 1, figsize=(6,6))

ax.scatter(X[Y==-1,0], X[Y==-1,1], s=20, c='C0', marker='x', label='-1')
ax.scatter(X[Y==1,0], X[Y==1,1], s=20, c='C1', marker='x', label='1')
ax.legend(('1','-1'), loc='best', title="Clases")
ax.set_xlabel('$X_{1}$', fontsize=14)
ax.set_ylabel('$X_{2}$', fontsize=14)


a = boundary(perceptron.W, -1)
b = boundary(perceptron.W, 1)

plt.plot([-1, 1], [a, b], 'r')

plt.grid(True)